# Preview the EAWAG scans
Let's see what we did there...

In [1]:
import platform
import os
import glob
import pandas
import imageio
import numpy
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import seaborn
import dask
import dask_image.imread
from dask.distributed import Client, LocalCluster
import skimage
from tqdm.auto import tqdm, trange

In [2]:
# Set dask temporary folder
# Do this before creating a client: https://stackoverflow.com/a/62804525/323100
import tempfile
if 'Linux' in platform.system():
    # Check if me mounted the FastSSD, otherwise go to standard tmp file
    if os.path.exists(os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')):
        tmp = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD', 'tmp')
    else:
        tmp = tempfile.gettempdir()
elif 'Darwin' in platform.system():
    tmp = tempfile.gettempdir()
else:
    if 'anaklin' in platform.node():
        tmp = os.path.join('F:\\tmp')
    else:
        tmp = os.path.join('D:\\tmp')
dask.config.set({'temporary_directory': tmp})
print('Dask temporary files go to %s' % dask.config.get('temporary_directory'))

Dask temporary files go to /media/habi/Fast_SSD/tmp


In [3]:
from dask.distributed import Client
client = Client()

2023-01-31 17:24:01,907 - distributed.diskutils - INFO - Found stale lock file and directory '/media/habi/Fast_SSD/tmp/dask-worker-space/worker-c98sv0wk', purging


In [4]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 32,Total memory: 125.80 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43691,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 125.80 GiB
Comm: tcp://127.0.0.1:45145,Total threads: 4
Dashboard: http://127.0.0.1:32931/status,Memory: 15.73 GiB
Nanny: tcp://127.0.0.1:35743,


In [5]:
print('You can seee what DASK is doing at "http://localhost:%s/status"' % client.scheduler_info()['services']['dashboard'])

You can seee what DASK is doing at "http://localhost:8787/status"


In [6]:
# # Ignore warnings in the notebook
# import warnings
# warnings.filterwarnings("ignore")

In [7]:
# Set up figure defaults
plt.rc('image', cmap='gray', interpolation='nearest')  # Display all images in b&w and with 'nearest' interpolation
plt.rcParams['figure.figsize'] = (16, 9)  # Size up figures a bit
plt.rcParams['figure.dpi'] = 200

In [8]:
# Setup scale bar defaults
plt.rcParams['scalebar.location'] = 'lower right'
plt.rcParams['scalebar.frameon'] = False
plt.rcParams['scalebar.color'] = 'white'

In [9]:
# Display all plots identically
lines = 3
# And then do something like
# plt.subplot(lines, int(numpy.ceil(len(Data) / float(lines))), c + 1)

In [10]:
# Different locations if running either on Linux or Windows
FastSSD = True
nanoct = True  # Load the data directly from the 2214
overthere = False  # Load the data directly from the iee-research_storage drive
# to speed things up significantly
if 'Linux' in platform.system():
    if FastSSD:
        BasePath = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')
    elif overthere:
        BasePath = os.path.join(os.sep, 'home', 'habi', 'research-storage-iee')
    elif nanoct:
        BasePath = os.path.join(os.path.sep, 'home', 'habi', '2214')
    else:
        BasePath = os.path.join(os.sep, 'home', 'habi', '1272')
elif 'Darwin' in platform.system():
    FastSSD = False
    BasePath = os.path.join('/Users/habi/Dev/EAWAG/Data')
elif 'Windows' in platform.system():
    if FastSSD:
        BasePath = os.path.join('F:\\')
    else:
        if 'ana' in platform.node():
            BasePath = os.path.join('\\\\resstore.unibe.ch', 'iee_aqua', 'microCTupload')
        else:
            BasePath = os.path.join('D:\\Results')
if overthere:
    Root = BasePath
else:
    Root = os.path.join(BasePath, 'EAWAG')
print('We are loading all the data from %s' % Root)

We are loading all the data from /media/habi/Fast_SSD/EAWAG


In [11]:
def get_pixelsize(logfile):
    """Get the pixel size from the scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Image Pixel' in line and 'Scaled' not in line:
                pixelsize = float(line.split('=')[1])
    return(pixelsize)

In [12]:
def get_projectionsize(logfile):
    """How big did we set the camera?"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Number Of Rows' in line:
                y = int(line.split('=')[1])
            if 'Number Of Columns' in line:
                x = int(line.split('=')[1])
    return(x * y)

In [13]:
def get_filter(logfile):
    """Get the filter we used whole scanning from the scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Filter=' in line:
                whichfilter = line.split('=')[1].strip()
    return(whichfilter)

In [14]:
def get_exposuretime(logfile):
    """Get the exposure time size from the scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Exposure' in line:
                exposuretime = int(line.split('=')[1])
    return(exposuretime)

In [15]:
def get_ringartefact(logfile):
    """Get the ring artefact correction from the  scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Ring Artifact' in line:
                ringartefactcorrection = int(line.split('=')[1])
    return(ringartefactcorrection)

In [16]:
def get_reconstruction_grayvalue(logfile):
    grayvalue = None
    """How did we map the brightness of the reconstructions?"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Maximum for' in line:
                grayvalue = float(line.split('=')[1])
    return(grayvalue)

In [17]:
def get_beamhardening(logfile):
    """Get the beamhardening correction from the  scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Hardening' in line:
                beamhardeningcorrection = int(line.split('=')[1])
    return(beamhardeningcorrection)

In [18]:
def get_rotationstep(logfile):
    """Get the rotation step from the scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Rotation Step' in line:
                rotstep = float(line.split('=')[1])
    return(rotstep)

In [19]:
def get_frameaveraging(logfile):
    """Get the frame averaging from the scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Averaging' in line:
                avg = line.split('=')[1]
    return(avg)

In [20]:
def get_defectpixelmasking(logfile):
    """Check the 'defect pixel masking' setting"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'defect pixel mask' in line:
                defectmask = int(line.split('=')[1].strip())
    return(defectmask)

In [21]:
def get_machine(logfile):
    """Get the machine we used to scan"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Scanner' in line:
                machine = line.split('=')[1].strip()
    return(machine)

In [22]:
def get_scantime(logfile):
    """How long did we scan?"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Scan duration' in line:
                time = line.split('=')[1].strip()
    return(pandas.to_timedelta(time))

In [23]:
def get_stacks(logfile):
    """How many stacks/connected scans did we make?"""
    stacks = 1
    with open(logfile, 'r') as f:
        for line in f:
            if 'conn' in line:
                stacks = int(line.split('=')[1])
    return(stacks)

In [24]:
def get_scandate(logfile, verbose=False):
    """When did we scan the fish?"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Study Date and Time' in line:
                if verbose:
                    print('Found "date" line: %s' % line.strip())
                datestring = line.split('=')[1].strip().replace('  ', ' ')
                if verbose:
                    print('The date string is: %s' % datestring)
                date = pandas.to_datetime(datestring, format='%d %b %Y %Hh:%Mm:%Ss')
                if verbose:
                    print('Parsed to: %s' % date)
    return(date.isoformat())

In [25]:
def get_git_hash():
    '''
    Get the current git hash from the repository.
    Based on http://stackoverflow.com/a/949391/323100 and
    http://stackoverflow.com/a/18283905/323100
    '''
    from subprocess import Popen, PIPE
    import os
    gitprocess = Popen(['git',
                        '--git-dir',
                        os.path.join(os.getcwd(), '.git'),
                        'rev-parse',
                        '--short',
                        '--verify',
                        'HEAD'],
                       stdout=PIPE)
    (output, _) = gitprocess.communicate()
    return output.strip().decode("utf-8")

In [26]:
# # Make directory for output
# OutPutDir = os.path.join(os.getcwd(), 'Output', get_git_hash())
# print('We are saving all the output to %s' % OutPutDir)
# os.makedirs(OutPutDir, exist_ok=True)

In [27]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [28]:
# Get *all* log files
# Using os.walk is way faster than using recursive glob.glob, see DataWrangling.ipynb for details
# Not sorting the found logfiles is also making it quicker
Data['LogFile'] = [os.path.join(root, name)
                   for root, dirs, files in os.walk(Root)
                   for name in files
                   if name.endswith((".log"))]

In [29]:
# Get all folders
Data['Folder'] = [os.path.dirname(f) for f in Data['LogFile']]

In [30]:
Data.sample(n=10)

,LogFile,Folder
1054,/media/habi/Fast_SSD/EAWAG/IG75/rec_pj/IG75_re...,/media/habi/Fast_SSD/EAWAG/IG75/rec_pj
1107,/media/habi/Fast_SSD/EAWAG/KC31/head/rec/KC31_...,/media/habi/Fast_SSD/EAWAG/KC31/head/rec
870,/media/habi/Fast_SSD/EAWAG/22478/proj/22478~02...,/media/habi/Fast_SSD/EAWAG/22478/proj
403,/media/habi/Fast_SSD/EAWAG/10715/head/proj/107...,/media/habi/Fast_SSD/EAWAG/10715/head/proj
157,/media/habi/Fast_SSD/EAWAG/103908/head/proj/TM...,/media/habi/Fast_SSD/EAWAG/103908/head/proj
433,/media/habi/Fast_SSD/EAWAG/10794/rec_rescan/10...,/media/habi/Fast_SSD/EAWAG/10794/rec_rescan
708,/media/habi/Fast_SSD/EAWAG/13115/head/proj/131...,/media/habi/Fast_SSD/EAWAG/13115/head/proj
134,/media/habi/Fast_SSD/EAWAG/103767/proj/103767~...,/media/habi/Fast_SSD/EAWAG/103767/proj
1117,/media/habi/Fast_SSD/EAWAG/MA31/head/proj/MA31...,/media/habi/Fast_SSD/EAWAG/MA31/head/proj
326,/media/habi/Fast_SSD/EAWAG/10618/proj/10618~01...,/media/habi/Fast_SSD/EAWAG/10618/proj


In [31]:
# Check for samples which are not yet reconstructed
for c, row in Data.iterrows():
    # Iterate over every 'proj' folder
    if 'proj' in row.Folder:
        if 'TScopy' not in row.Folder and 'PR' not in row.Folder:
            # If there's nothing with 'rec*' on the same level, then tell us
            if not glob.glob(row.Folder.replace('proj', '*rec*')):
                print('- %s is missing matching reconstructions' % row.LogFile[len(Root) + 1:])

- 103761/proj_oj/103761.log is missing matching reconstructions


In [32]:
# 103375/proj_stuck/*.log is a scan where we lost air pressure in the building and the rotation stage got stuck. We renamed the folder afterwards.
# 103761/proj_oj/103761.log is a scan where the sample holder touched the source

In [33]:
# Search for any .csv files in each folder
Data['XYAlignment'] = [glob.glob(os.path.join(f, '*T*.csv')) for f in Data['Folder']]

In [34]:
# Display samples which are missing the .csv-files for the XY-alignment
for c, row in Data.iterrows():
    # Iterate over every 'proj' folder
    if 'proj' in row['Folder']:
        if not row['XYAlignment']:
            if not any(x in row.LogFile for x in ['rectmp.log',  # because we only exclude temporary logfiles in a later step
                                                  'proj_nofilter',  # since these two scans of single teeth don't contain a reference scan
                                                  'TScopy',  # discard *t*hermal *s*hift data
                                                  os.path.join('TJ3', 'jaw_v1'),  # no reference scan
                                                  os.path.join('28', 'full_188um'),  # no reference scan
                                                  os.path.join('75', 'proj_stuck'),  # 103375\proj_stuck which got stuck
                                                  os.path.join('106985', 'proj'),  # 106985/proj is a b0rked scan where something went wrong with the exposure time
                                                  os.path.join('161543', 'head_30'),  # 161543\head_30um has no reference scan
                                                  os.path.join('21322', 'jaw'),  # two scans of 21322 which have no reference scan
                                                  os.path.join('21322', 'whole'),  # two scans of 21322 which have no reference scan
                                                  os.path.join('31', 'moved_proj'),  # MA31\moved_proj which moved during acquisition
                                                  os.path.join('95', 'proj', '14295'),  # no reference scan
                                                  os.path.join('104061', 'head', 'proj_pressure')]):   # We lost air pressure (in the building) during this scan
                print('- %s has *not* been X/Y aligned' % row.LogFile[len(Root) + 1:])

In [35]:
# Get rid of all non-rec logfiles
for c, row in Data.iterrows():
    if 'rec' not in row.Folder:
        Data.drop([c], inplace=True)
    elif 'SubScan' in row.Folder:
        Data.drop([c], inplace=True)
    elif 'rectmp.log' in row.LogFile:
        Data.drop([c], inplace=True)
# Reset dataframe to something that we would get if we only would have loaded the 'rec' files
Data = Data.reset_index(drop=True)

In [36]:
# Generate us some meaningful colums
Data['Fish'] = [l[len(Root) + 1:].split(os.sep)[0] for l in Data['LogFile']]
Data['Scan'] = ['_'.join(l[len(Root) + 1:].split(os.sep)[1:-1]) for l in Data['LogFile']]

In [37]:
Data.tail()

,LogFile,Folder,XYAlignment,Fish,Scan
366,/media/habi/Fast_SSD/EAWAG/ZU12/rec_reconstruc...,/media/habi/Fast_SSD/EAWAG/ZU12/rec_reconstruct,[],ZU12,rec_reconstruct
367,/media/habi/Fast_SSD/EAWAG/ZuOS115/head/rec/Zu...,/media/habi/Fast_SSD/EAWAG/ZuOS115/head/rec,[],ZuOS115,head_rec
368,/media/habi/Fast_SSD/EAWAG/ZuOS115/rec/ZuOS115...,/media/habi/Fast_SSD/EAWAG/ZuOS115/rec,[],ZuOS115,rec
369,/media/habi/Fast_SSD/EAWAG/ZuOS115/rec_oj/ZuOS...,/media/habi/Fast_SSD/EAWAG/ZuOS115/rec_oj,[],ZuOS115,rec_oj
370,/media/habi/Fast_SSD/EAWAG/ZuOS148/rec/ZuOS148...,/media/habi/Fast_SSD/EAWAG/ZuOS148/rec,[],ZuOS148,rec


In [38]:
# Get the file names of the reconstructions, unsorted
Data['Filenames Reconstructions'] = [glob.glob(os.path.join(f, '*rec0*.png')) for f in Data['Folder']]
# How many reconstructions do we have?
Data['Number of reconstructions'] = [len(r) for r in Data['Filenames Reconstructions']]

In [39]:
# Drop samples which have either not been reconstructed yet or of which we deleted the reconstructions with
# `find . -name "*rec*.png" -type f -mtime +333 -delete`
# Based on https://stackoverflow.com/a/13851602
# for c,row in Data.iterrows():
#     if not row['Number of reconstructions']:
#         print('%s contains no PNG files, we might be currently reconstructing it' % row.Folder)
Data = Data[Data['Number of reconstructions'] > 0]
Data.reset_index(drop=True, inplace=True)
print('We have %s folders with reconstructions' % (len(Data)))

We have 111 folders with reconstructions


In [40]:
# Get parameters to doublecheck from logfiles
Data['Voxelsize'] = [get_pixelsize(log) for log in Data['LogFile']]
Data['Filter'] = [get_filter(log) for log in Data['LogFile']]
Data['Exposuretime'] = [get_exposuretime(log) for log in Data['LogFile']]
Data['Scanner'] = [get_machine(log) for log in Data['LogFile']]
Data['Averaging'] = [get_frameaveraging(log) for log in Data['LogFile']]
Data['ProjectionSize'] = [get_projectionsize(log) for log in Data['LogFile']]
Data['RotationStep'] = [get_rotationstep(log) for log in Data['LogFile']]
Data['CameraWindow'] = [round((ps ** 0.5) / 100) * 100 for ps in Data['ProjectionSize']]
Data['Grayvalue'] = [get_reconstruction_grayvalue(log) for log in Data['LogFile']]
Data['RingartefactCorrection'] = [get_ringartefact(log) for log in Data['LogFile']]
Data['BeamHardeningCorrection'] = [get_beamhardening(log) for log in Data['LogFile']]
Data['DefectPixelMasking'] = [get_defectpixelmasking(log) for log in Data['LogFile']]
Data['Scan date'] = [get_scandate(log) for log in Data['LogFile']]
Data['Scan time'] = [get_scantime(log) for log in Data['LogFile']]
Data['Stacks'] = [get_stacks(log) for log in Data['LogFile']]

In [41]:
# Check ringremoval parameters
for machine in Data['Scanner'].unique():
    print('For the %s we have '
          'ringartefact-correction values of %s' % (machine,
                                                    Data[Data.Scanner == machine]['RingartefactCorrection'].unique()))

For the SkyScan2214 we have ringartefact-correction values of [0]
For the SkyScan1272 we have ringartefact-correction values of [13 14]


In [42]:
for rac in sorted(Data['RingartefactCorrection'].unique()):
    print('Ringartefact-correction %02s is found in %03s scans' % (rac,
                                                                   Data[Data.RingartefactCorrection == rac]['RingartefactCorrection'].count()))

Ringartefact-correction  0 is found in 108 scans
Ringartefact-correction 13 is found in   2 scans
Ringartefact-correction 14 is found in   1 scans


Some consistency checks

In [43]:
# seaborn.scatterplot(data=Data, x='Fish', y='RingartefactCorrection', hue='Scanner')
# plt.title('Ringartefact correction')
# plt.show()

In [44]:
# Display ringremoval parameters
for scanner in Data.Scanner.unique():
    print('----', scanner, '----')
    for c, row in Data[Data.Scanner == scanner].iterrows():
        if row.RingartefactCorrection != 0:
            print('Fish %s scan %s was reconstructed with RAC of %s' % (row.Fish,
                                                                        row.Scan,
                                                                        row.RingartefactCorrection))

---- SkyScan2214 ----
---- SkyScan1272 ----
Fish 21322 scan whole_rec_full was reconstructed with RAC of 13
Fish 106985 scan rec was reconstructed with RAC of 14
Fish IG92 scan rec was reconstructed with RAC of 13


In [45]:
# Check beamhardening parameters
for scanner in Data.Scanner.unique():
    print('For the %s we have '
          'beamhardening correction values of %s' % (scanner,
                                                     Data[Data.Scanner == scanner]['BeamHardeningCorrection'].unique()))

For the SkyScan2214 we have beamhardening correction values of [0]
For the SkyScan1272 we have beamhardening correction values of [0]


In [46]:
# Display beamhardening parameters
for scanner in Data.Scanner.unique():
    print('----', scanner, '----')
    for c, row in Data[Data.Scanner == scanner].iterrows():
        if row.BeamHardeningCorrection != 0:
            print('Scan %s of fish %s was reconstructed with beam hardening correction of %s' % (row.Scan,
                                                                                                 row.Fish,
                                                                                                 row.BeamHardeningCorrection))

---- SkyScan2214 ----
---- SkyScan1272 ----


In [47]:
# seaborn.scatterplot(data=Data,
#                     x='Fish',
#                     y='BeamHardeningCorrection',
#                     hue='Scanner')
# plt.title('Beamhardening correction')
# plt.show()

In [48]:
# Check defect pixel masking parameters
for scanner in Data.Scanner.unique():
    print('For the %s we have '
          'defect pixel masking values of %s' % (scanner,
                                                 Data[Data.Scanner == scanner]['DefectPixelMasking'].unique()))

For the SkyScan2214 we have defect pixel masking values of [0]
For the SkyScan1272 we have defect pixel masking values of [50  0]


In [49]:
for dpm in sorted(Data['DefectPixelMasking'].unique()):
    print('A defect pixel masking of %02s is found in %03s scans' % (dpm,
                                                                     Data[Data.DefectPixelMasking == dpm]['DefectPixelMasking'].count()))

A defect pixel masking of  0 is found in 109 scans
A defect pixel masking of 50 is found in   2 scans


In [50]:
# seaborn.scatterplot(data=Data, x='Fish', y='DefectPixelMasking', hue='Scanner')
# plt.title('Defect pixel masking')
# plt.show()

In [51]:
# Display defect pixel masking parameters
for scanner in Data.Scanner.unique():
    print('----', scanner, '----')
    for c, row in Data[Data.Scanner == scanner].iterrows():
        if row.Scanner == 'SkyScan1272' and row.DefectPixelMasking != 50:
            print('Fish %s scan %s was reconstructed with DPM of %s' % (row.Fish,
                                                                        row.Scan,
                                                                        row.DefectPixelMasking))
        if row.Scanner == 'SkyScan2214' and row.DefectPixelMasking != 0:
            print('Fish %s scan %s was reconstructed with DPM of %s' % (row.Fish,
                                                                        row.Scan,
                                                                        row.DefectPixelMasking))

---- SkyScan2214 ----
---- SkyScan1272 ----
Fish 106985 scan rec was reconstructed with DPM of 0


Check and display scan times

In [52]:
Data['Scan time total'] = [st * stk for st, stk in zip(Data['Scan time'], Data['Stacks'])]

In [53]:
# # https://www.geeksforgeeks.org/iterating-over-rows-and-columns-in-pandas-dataframe/
# columns = list(Data)
# columns.remove('Folder')
# columns.remove('Fish')
# columns.remove('LogFile')
# columns.remove('Filenames Reconstructions')
# columns.remove('Number of reconstructions')
# columns.remove('Grayvalue')
# columns.remove('Scan time')
# columns.remove('Scan time total')
# columns.remove('Scan date')
# print(columns)
# for col in columns:
#     print(col)
#     print(Data[col].unique())
#     print(80 * '-')

In [54]:
# # Check voxel sizes (*rounded* to two after-comma values)
# # If different, spit out which values
# roundto = 2
# if len(Data['Voxelsize'].round(roundto).unique()) > 1:
#     print('We scanned all datasets with %s different voxel sizes' % len(Data['Voxelsize'].round(roundto).unique()))
#     for vs in sorted(Data['Voxelsize'].round(roundto).unique()):
#         print('-', vs, 'um for ', end='')
#         for c, row in Data.iterrows():
#             if float(vs) == round(row['Voxelsize'], roundto):
#                 print(os.path.join(row['Fish'], row['Scan']), end=', ')
#         print('')
# else:
#     print('We scanned all datasets with equal voxel size, namely %s um.' % float(Data['Voxelsize'].round(roundto).unique()))

In [55]:
# if len(Data['Grayvalue'].unique()) > 1:
#     print('We reconstructed the datasets with different maximum gray values, namely')
#     for gv in Data['Grayvalue'].unique():
#         print(gv, 'for Samples ', end='')
#         for c, row in Data.iterrows():
#             if float(gv) == row['Grayvalue']:
#                 print(os.path.join(row['Fish'], row['Scan']), end=', ')
#         print('')
# else:
#     print('We reconstructed all datasets with equal maximum gray value, namely %s.' % Data['Grayvalue'].unique()[0])

In [56]:
# Data[['Fish', 'Scan',
#       'Voxelsize', 'Scanner',
#       'Scan date', 'CameraWindow', 'RotationStep', 'Averaging',
#       'Scan time', 'Stacks', 'Scan time total']]

In [57]:
# Get an overview over the total scan time
# Nice output based on https://stackoverflow.com/a/8907407/323100
total_seconds = int(Data['Scan time total'].sum().total_seconds())
hours, remainder = divmod(total_seconds, 60 * 60)
minutes, seconds = divmod(remainder, 60)
print('In total, we scanned for %s hours and %s minutes)' % (hours, minutes))
for machine in Data['Scanner'].unique():
    total_seconds = int(Data[Data['Scanner'] == machine]['Scan time total'].sum().total_seconds())
    hours, remainder = divmod(total_seconds, 60 * 60)
    minutes, seconds = divmod(remainder, 60)
    print('\t - Of these, we scanned %s hours and %s minutes on the %s,'
          ' for %s scans' % (hours,
                             minutes,
                             machine,
                             len(Data[Data['Scanner'] == machine])))

In total, we scanned for 239 hours and 56 minutes)
	 - Of these, we scanned 215 hours and 27 minutes on the SkyScan2214, for 108 scans
	 - Of these, we scanned 24 hours and 29 minutes on the SkyScan1272, for 3 scans


In [58]:
# Sort our dataframe by scan date
# Data.sort_values(by='Scan date')

In [59]:
Data.tail()

,LogFile,Folder,XYAlignment,Fish,Scan,Filenames Reconstructions,Number of reconstructions,Voxelsize,Filter,Exposuretime,...,RotationStep,CameraWindow,Grayvalue,RingartefactCorrection,BeamHardeningCorrection,DefectPixelMasking,Scan date,Scan time,Stacks,Scan time total
106,/media/habi/Fast_SSD/EAWAG/TJ2/head/rec/TJ2_re...,/media/habi/Fast_SSD/EAWAG/TJ2/head/rec,[],TJ2,head_rec,[/media/habi/Fast_SSD/EAWAG/TJ2/head/rec/TJ2_r...,1746,15.000176,No Filter,790,...,0.1,2400,0.104671,0,0,0,2022-04-25T11:12:33,0 days 01:50:40,1,0 days 01:50:40
107,/media/habi/Fast_SSD/EAWAG/TJ3/head/rec/TJ3_re...,/media/habi/Fast_SSD/EAWAG/TJ3/head/rec,[],TJ3,head_rec,[/media/habi/Fast_SSD/EAWAG/TJ3/head/rec/TJ3_r...,1906,13.001449,No Filter,612,...,0.2,1200,0.154967,0,0,0,2022-01-24T15:20:14,0 days 00:33:09,3,0 days 01:39:27
108,/media/habi/Fast_SSD/EAWAG/TMG15/head/rec/TMG1...,/media/habi/Fast_SSD/EAWAG/TMG15/head/rec,[],TMG15,head_rec,[/media/habi/Fast_SSD/EAWAG/TMG15/head/rec/TMG...,1862,17.999448,No Filter,655,...,0.1,2400,0.123096,0,0,0,2022-03-10T14:22:44,0 days 01:40:16,1,0 days 01:40:16
109,/media/habi/Fast_SSD/EAWAG/TS03/head/rec/TS03_...,/media/habi/Fast_SSD/EAWAG/TS03/head/rec,[],TS03,head_rec,[/media/habi/Fast_SSD/EAWAG/TS03/head/rec/TS03...,1638,21.999264,No Filter,748,...,0.2,1200,0.103910,0,0,0,2022-03-08T14:29:25,0 days 00:39:18,2,0 days 01:18:36
110,/media/habi/Fast_SSD/EAWAG/ZuOS115/head/rec/Zu...,/media/habi/Fast_SSD/EAWAG/ZuOS115/head/rec,[],ZuOS115,head_rec,[/media/habi/Fast_SSD/EAWAG/ZuOS115/head/rec/Z...,1843,17.999448,No Filter,655,...,0.1,2400,0.074241,0,0,0,2022-03-11T11:02:55,0 days 01:38:46,1,0 days 01:38:46


In [60]:
# Data['Filename PreviewImage'] = [sorted(glob.glob(os.path.join(f, '*_spr.bmp')))[0] for f in Data['Folder']]
# Data['PreviewImage'] = [dask_image.imread.imread(pip).squeeze()
#                         if pip
#                         else numpy.random.random((100, 100)) for pip in Data['Filename PreviewImage']]

In [61]:
# Make an approximately square overview image
# lines = 10

In [62]:
# for c, row in Data.iterrows():
#     plt.subplot(lines, int(numpy.ceil(len(Data) / float(lines))), c + 1)
#     plt.imshow(row.PreviewImage.squeeze())
#     plt.title(os.path.join(row['Fish'], row['Scan']))
#     plt.gca().add_artist(ScaleBar(row['Voxelsize'],
#                                   'um',
#                                   color='black',
#                                   frameon=True))
#     plt.axis('off')
# plt.tight_layout()
# plt.savefig(os.path.join(Root, 'ScanOverviews.png'),
#             bbox_inches='tight')
# plt.show()

In [63]:
# # Load all reconstructions DASK arrays
# Reconstructions = [dask_image.imread.imread(os.path.join(folder,'*rec*.png')) for folder in Data['Folder']]
# Load all reconstructions into ephemereal DASK arrays, with a nice progress bar...
Reconstructions = [None] * len(Data)
for c, row in tqdm(Data.iterrows(),
                   desc='Loading reconstructions',
                   total=len(Data)):
    Reconstructions[c] = dask_image.imread.imread(os.path.join(row['Folder'], '*rec*.png'))

Loading reconstructions:   0%|          | 0/111 [00:00<?, ?it/s]

In [64]:
# Check if something went wrong
# for file in Data['OutputNameRec']:
#     print(file)
#     dask.array.from_zarr(file)

In [65]:
# How big are the datasets?
Data['Size'] = [rec.shape for rec in Reconstructions]

In [66]:
# The three cardinal directions
# Names adapted to fishes: https://en.wikipedia.org/wiki/Fish_anatomy#Body
directions = ['Anteroposterior',
              'Lateral',
              'Dorsoventral']

In [67]:
# # # Get rid of certain samples
# # # This is nice to have if we're running into troubles with certain samples/scans and have to re-reconstruct them
# # # In this case we don't have to re-read in *all* the log files and reconstructions...
# for c, row in Data.iterrows():
#     if '103761' in row.Fish:
#         Data.drop([c], inplace=True)
# # Reset dataframe to something that we would get if we only would have loaded the 'rec' files
# Data = Data.reset_index(drop=True)

In [ ]:
# Read or calculate the middle slices, put them into the dataframe and save them to disk
for d, direction in enumerate(directions):
    Data['Mid_' + direction] = ''
for c, row in tqdm(Data.iterrows(), desc='Working on central images', total=len(Data)):
    for d, direction in tqdm(enumerate(directions),
                             desc='%s/%s' % (row['Fish'], row['Scan']),
                             leave=False,
                             total=len(directions)):
        outfilepath = os.path.join(os.path.dirname(row['Folder']),
                                   '%s.%s.Center.%s.png' % (row['Fish'], row['Scan'], direction))
        if os.path.exists(outfilepath):
            try:
                Data.at[c, 'Mid_' + direction] = dask_image.imread.imread(outfilepath)
            except:
                print('Something went wrong with %s/%s %s' % (row['Fish'], row['Scan'], direction))
                print('Try to delete the %s and restart the notebook' % outfilepath)
        else:
            # Generate requested axial view
            if 'Anteroposterior' in direction:
                Data.at[c, 'Mid_' + direction] = Reconstructions[c][row.Size[0] // 2].compute()
            if 'Lateral' in direction:
                Data.at[c, 'Mid_' + direction] = Reconstructions[c][:, row.Size[1] // 999992, :].compute()
            if 'Dorsoventral' in direction:
                Data.at[c, 'Mid_' + direction] = Reconstructions[c][:, :, row.Size[2] // 2].compute()
            # Save the calculated 'direction' view to disk
            imageio.imwrite(outfilepath, (Data.at[c, 'Mid_' + direction]))

Working on central images:   0%|          | 0/111 [00:00<?, ?it/s]

10151/head_rec:   0%|          | 0/3 [00:00<?, ?it/s]

103375/head_rec:   0%|          | 0/3 [00:00<?, ?it/s]

103571/head_rec:   0%|          | 0/3 [00:00<?, ?it/s]

Something went wrong with 103571/head_rec Dorsoventral
Try to delete the /media/habi/Fast_SSD/EAWAG/103571/head/103571.head_rec.Center.Dorsoventral.png and restart the notebook


/home/habi/miniconda3/envs/eawag/lib/python3.10/site-packages/pims/api.py:204: UserWarning: <class 'pims.image_reader.ImageReader'> errored: Could not load "" 
Reason: "image file is truncated"
Please see documentation at: http://pillow.readthedocs.io/en/latest/installation.html#external-libraries
  warn(message)
/home/habi/miniconda3/envs/eawag/lib/python3.10/site-packages/pims/api.py:204: UserWarning: <class 'pims.image_reader.ImageReaderND'> errored: Could not load "" 
Reason: "image file is truncated"
Please see documentation at: http://pillow.readthedocs.io/en/latest/installation.html#external-libraries
  warn(message)
/home/habi/miniconda3/envs/eawag/lib/python3.10/site-packages/pims/api.py:204: UserWarning: <class 'pims.imageio_reader.ImageIOReader'> errored: Could not load "" 
Reason: "image file is truncated"
Please see documentation at: http://pillow.readthedocs.io/en/latest/installation.html#external-libraries
  warn(message)


103635/head_rec:   0%|          | 0/3 [00:00<?, ?it/s]

103704/head_rec:   0%|          | 0/3 [00:00<?, ?it/s]

103718/head_rec:   0%|          | 0/3 [00:00<?, ?it/s]

103723/head_rec:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Show/save middle slices
for c, row in tqdm(Data.iterrows(),
                   desc='Saving overview of central images',
                   total=len(Data)):
    outfilepath = os.path.join(os.path.dirname(row['Folder']),
                               '%s.%s.CentralSlices.png' % (row['Fish'], row['Scan']))
    if not os.path.exists(outfilepath):
        for d, direction in tqdm(enumerate(directions),
                                 desc='%s/%s' % (row['Fish'], row['Scan']),
                                 leave=False,
                                 total=len(directions)):
            plt.subplot(1, 3, d + 1)
            plt.imshow(row['Mid_' + direction].squeeze())
            if d == 0:
                plt.axhline(row.Size[1] // 2, c=seaborn.color_palette()[0])
                plt.axvline(row.Size[2] // 2, c=seaborn.color_palette()[1])
                plt.gca().add_artist(ScaleBar(row['Voxelsize'],
                                              'um',
                                              color=seaborn.color_palette()[2]))
            elif d == 1:
                plt.axhline(row.Size[0] // 2, c=seaborn.color_palette()[2])
                plt.axvline(row.Size[2] // 2, c=seaborn.color_palette()[1])
                plt.gca().add_artist(ScaleBar(row['Voxelsize'],
                                              'um',
                                              color=seaborn.color_palette()[0]))
            else:
                plt.axhline(row.Size[0] // 2, c=seaborn.color_palette()[2])
                plt.axvline(row.Size[1] // 2, c=seaborn.color_palette()[0])
                plt.gca().add_artist(ScaleBar(row['Voxelsize'],
                                              'um',
                                              color=seaborn.color_palette()[1]))
            plt.title('%s\nCentral %s slice' % (os.path.join(row['Fish'], row['Scan']), direction.lower()))
            plt.axis('off')
        plt.savefig(outfilepath,
                    transparent=True,
                    bbox_inches='tight')
        plt.show()

In [ ]:
# Read or calculate the directional MIPs, put them into the dataframe and save them to disk
for d, direction in enumerate(directions):
    Data['MIP_' + direction] = ''
for c, row in tqdm(Data.iterrows(), desc='Working on MIPs', total=len(Data)):
    for d, direction in tqdm(enumerate(directions),
                             desc='%s/%s' % (row['Fish'], row['Scan']),
                             leave=False,
                             total=len(directions)):
        outfilepath = os.path.join(os.path.dirname(row['Folder']),
                                   '%s.%s.MIP.%s.png' % (row['Fish'], row['Scan'], direction))
        if os.path.exists(outfilepath):
            try:
                Data.at[c, 'MIP_' + direction] = dask_image.imread.imread(outfilepath).squeeze()
            except:
                print('Something went wrong with %s/%s %s' % (row['Fish'], row['Scan'], direction))
                print('Try to delete the %s and restart the notebook' % outfilepath)
                pass
        else:
            # Generate MIP
            Data.at[c, 'MIP_' + direction] = Reconstructions[c].max(axis=d).compute().squeeze()
            # Save it out
            imageio.imwrite(outfilepath, Data.at[c, 'MIP_' + direction].astype('uint8'))

In [ ]:
print('We have %s reconstructions on %s' % (Data['Number of reconstructions'].sum(), Root))

In [ ]:
print('This is about %s reconstructions per scan (%s scans in %s folders)' % (round(Data['Number of reconstructions'].sum() / len(Data)),
                                                                              len(Data),
                                                                              len(Data.Fish.unique())))

In [ ]:
# Show/save MIP slices
for c, row in tqdm(Data.iterrows(),
                   desc='Saving overview of MIP images',
                   total=len(Data)):
    outfilepath = os.path.join(os.path.dirname(row['Folder']),
                               '%s.%s.MIPs.png' % (row['Fish'], row['Scan']))
    if not os.path.exists(outfilepath):
        for d, direction in tqdm(enumerate(directions),
                                 desc='%s/%s' % (row['Fish'], row['Scan']),
                                 leave=False,
                                 total=len(directions)):
            plt.subplot(1, 3, d + 1)
            plt.imshow(row['MIP_' + direction])
            plt.gca().add_artist(ScaleBar(row['Voxelsize'],
                                          'um'))
            plt.title('%s MIP' % direction)
            plt.axis('off')
            plt.title('%s\n%s MIP' % (os.path.join(row['Fish'], row['Scan']), direction))
            plt.savefig(outfilepath,
                        transparent=True,
                        bbox_inches='tight')
        plt.show()

In [ ]:
# Calculate the histograms of one of the MIPs
# Caveat: dask.da.histogram returns histogram AND bins, making each histogram a 'nested' list of [h, b]
Data['Histogram'] = [dask.array.histogram(dask.array.array(mip),
                                          bins=2**8,
                                          range=[0, 2**8]) for mip in Data['MIP_Lateral']]
# Actually compute the data and put only h into the dataframe, so we can use it below.
# Discard the bins
Data['Histogram'] = [h for h, b in Data['Histogram']]

In [ ]:
def overeexposecheck(item, threshold=250, howmanypercent=0.1, whichone='Lateral', verbose=False):
    '''Function to check if a certain amount of voxels are brighter than a certain value'''
    if (Data['MIP_%s' % whichone][item] > threshold).sum() > (Data['MIP_%s' % whichone][item].size * howmanypercent / 100):
        if verbose:
            plt.imshow(Data['MIP_%s' % whichone][item])
            plt.imshow(numpy.ma.masked_equal(Data['MIP_%s' % whichone][item] > threshold, False),
                       cmap='viridis_r',
                       alpha=.618)
            plt.title('%s/%s\n%s of %s Mpixels (>%s%%) are brighter '
                      'than %s' % (Data['Fish'][item],
                                   Data['Scan'][item],
                                   (Data['MIP_%s' % whichone][item] > threshold).sum(),
                                   round(1e-6 * Data['MIP_%s' % whichone][item].size, 2),
                                   howmanypercent,
                                   threshold))
            plt.axis('off')
            plt.gca().add_artist(ScaleBar(Data['Voxelsize'][item],
                                          'um'))
            plt.show()
        return(True)
    else:
        return(False)

In [ ]:
# Check if 'too much' of the MIP is overexposed
# TODO: How much is 'too much'?
Data['OverExposed'] = [overeexposecheck(c,
                                        whichone='Lateral',
                                        verbose=True) for c, row in Data.iterrows()]

In [ ]:
for c, row in sorted(Data.iterrows()):
    plt.semilogy(row.Histogram,
                 label='%s' % os.path.join(row.Fish, row.Scan))
plt.xlim([0, 255])
plt.legend()
plt.show()

In [ ]:
print('At the moment, we have previewed %s scans of %s folders in %s' % (len(Data),
                                                                         len(Data.Fish.unique()),
                                                                         Root))